# Autonomous Class Attendance System using Face Recogsition

Importing modules

In [1]:
#for mathematical and matrix calculations, used by openCV
import numpy as np

#for image racognization
import cv2

#to interact with data frames and csv files
import pandas as pd

#to get the system time
import datetime

#to manipulate the processing speed and pattern
import time

# Modifying the database

In [2]:
def exportStu(x):
    
    #get the previous data
    att=pd.read_csv("attendance.csv")
    #adding the data to the data frame
    m=att.append(x,ignore_index=True)
    #writting back the updated data to the csv databse
    m.to_csv("attendance.csv",index=False)
    
    #fromatting the dictionary to fir the data frame
    for i in x:
        x[i]=[x[i]]
    
    #converting the dictionary to dataframe
    x=pd.DataFrame.from_dict(x)
    #printing today's attendance report
    print("Today's Report \n",x)
    
    #returning today's data as well as historical data
    return m,x
    
    

In [3]:
def update(x,y):
    #creating a blank dictionary to update the data frame
    attend={}
    
    #putting the recognised persons (images) in the attendance register
    for i in y:
        
        #assigning the first coloum as date components
        #we have used "Dummy" value to lavel the index no and roll no
        if i=='Dummy':
            date=datetime.datetime.now()
            d=str(date.day)+":"+str(date.month)+":"+str(date.year)+"  "+str(date.hour)+":"+str(date.minute)
            attend["Date"]=d
            
            #to skeep the firts iteration to assign date
            continue    
        
        #as index is same as roll no
        j=y.index(i)
        
        #1: present 0: absent
        if i in x:
            attend[str(j)]=1
        else:
            attend[str(j)]=0
    
    #printing acknowledgement with some dramatic effect
    time.sleep(1)
    print("Updation Complete.",end="")
    time.sleep(1)
    print(".",end="")
    time.sleep(0.7)
    print(".",end="")
    time.sleep(0.6)
    print(".",end="")
    time.sleep(0.5)
    print(".",end="")
    time.sleep(0.25)
    print(".")
    
    #returning today's data as well as historical data
    return exportStu(attend)

Attendance keeper
-----------------

In [4]:
#this will start the process of attendance taking
def start():
    #getting the cascade information from openCV
    faceDetect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    #getting the image source from web camera
    cam=cv2.VideoCapture(0)
    rec=cv2.face.LBPHFaceRecognizer_create()
    
    #declaring font of the text in the frame
    font=cv2.FONT_HERSHEY_COMPLEX
    
    #get the student list
    y=pd.read_csv("roll.csv")
    stuList=y["Name"].values.tolist()

    #reading the training data
    rec.read("recognizer/trainingData.yml")
    
    #initializing id value
    id=0
    
    #initializing an empty set to contain the student name 
    a=set()
    
    #running the image capturing window
    while(True):
        
        #reading the frame
        rett,img=cam.read()
        
        #convering the color image into grayscale as the cascade only understand grayscale value
        gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        #getting the multiscale value
        faces=faceDetect.detectMultiScale(gray,1.3,5)
        
        #putting a rectangle frame on face 
        for(x,y,w,h) in faces:
            
            #plotting the rectangle
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
            #predicting student id from the face in rectangle
            id,conf=rec.predict(gray[y:y+h,x:x+w])
            
            #getting student name from student id
            id=stuList[id]
            
            #putting the name in the rectangle
            cv2.putText(img,str(id),(x,y+h),font,0.55,(0,255,0),1)
        
        #starting webcam
        cv2.imshow("Face",img)
        
        #updating the attened student list
        a.add(id)
        
        #stopping camera
        if (cv2.waitKey(1)==ord("q")):
            break
    
    #closing camera window
    cam.release()
    cv2.destroyWindow("Face")
    
    #printing attended student list
    print("Student Presnt :",a)
    
    #returning today's data as well as historical data
    return update(a,stuList)

# Running the Autonomous Attendance Taker

Instructions:
----------------

1. To start the system call : start()
2. It will return two values (respectively):
        i. All historical data in a data frame
        ii. Attendance sheet of today's class

In [5]:
all,today=start()


Student Presnt : {'sid'}
Updation Complete......
Today's Report 
              Date  1
0  7:3:2019  11:7  1


In [6]:
all

,Date,1
0,7:3:2019 11:7,1.0
